# 합성곱 신경망

이 노트북을 주피터 노트북 뷰어(nbviewer.jupyter.org)로 보거나 구글 코랩(colab.research.google.com)에서 실행할 수 있습니다.

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://nbviewer.jupyter.org/github/rickiepark/nn-in-tf/blob/master/convolution_network.ipynb"><img src="https://jupyter.org/assets/main-logo.svg" width="28" />주피터 노트북 뷰어로 보기</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/rickiepark/nn-in-tf/blob/master/convolution_network.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />구글 코랩(Colab)에서 실행하기</a>
  </td>
</table>

## 패션 MNIST 데이터셋 적재

이 예제에서는 패션 MNIST 데이터셋을 사용합니다. MNIST 데이터셋은 손으로 쓴 숫자 이미지로 머신러닝 분야의 '헬로 월드' 데이터셋으로 사용되어 왔습니다. 패션 MNIST 데이터셋은 MNIST 데이터셋과 동일한 크기를 가진 흑백이미지로 구성되어 있습니다. 하지만 숫자가 아니라 10개의 의류를 분류하는 문제(classification)입니다. 최근에 MNIST 데이터셋 대신에 널리 사용되고 있습니다.

패션 MNIST 데이터셋도 텐서플로에 포함되어 있습니다. 먼저 텐서플로에서 케라스를 임포트합니다.

In [ ]:
from tensorflow import keras

`datasets`의 다른 함수와 마찬가지로 `datasets.fashion_mnist.load_data()` 함수는 훈련 세트와 테스트 세트를 넘파이 배열로 반환합니다.

`load_data()` 함수는 전체 데이터에서 20%를 테스트 세트로 떼어내어 반환합니다. y_train과 y_test는 맞추어야 할 의류를 나타내는 정수 값입니다.

In [ ]:
(x_train, y_train), (x_test, y_test) = keras.datasets.fashion_mnist.load_data()

사용할 훈련 세트의 크기와 테스트 세트의 크기를 확인해 보겠습니다.

In [ ]:
print(x_train.shape, y_train.shape)

훈련 세트는 60,000개의 이미지로 이루어져 있습니다. 이미지의 높이와 너비는 28x28 크기입니다. `x_train`의 첫 번째 샘플을 출력해 보겠습니다.

In [ ]:
print(x_train[0])

출력된 값은 각 픽셀의 0~255 사이의 흑백 강도입니다. 컬러 이미지이라면 RGB 채널이 있기 때문에 3차원 배열로 구성됩니다. 첫 번째 샘플을 맷플롭립 패키지를 사용해 이미지로 출력해 보겠습니다.

`imshow()` 함수는 배열 값을 받아 이미지로 출력합니다. 패션 MNIST는 흑백 이미지이므로 `cmap` 매개변수에 `'gray'`로 지정했습니다.

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.imshow(x_train[0], cmap='gray')

타깃 값 `y_train`은 샘플이 속한 클래스(class)를 나타내는 정수입니다. 이를 레이블(label)이라고 부릅니다. 처음 5개의 레이블을 출력해 보겠습니다.

In [ ]:
print(y_train[0:5])

`fashion_mnist.load_data()` 함수에서는 이 정수가 어떤 의미인지 알려 주지 않습니다. 레이블 값을 사람이 알 수 있는 범주로 바꾸기 위해 수동으로 클래스 목록을 만들어 보겠습니다.

In [ ]:
class_names = ['티셔츠/윗도리', '바지', '스웨터', '드레스', '코트', 
               '샌들', '셔츠', '스니커즈', '가방', '앵클부츠']

그 다음 첫 번째 샘플의 클래스 이름을 출력해 봅니다. `y_train` 배열의 값을 `class_names` 리스트의 인덱스로 사용하면 됩니다.

In [ ]:
print(class_names[y_train[0]])

다른 샘플도 확인해 보세요.

이제 테스트 세트의 크기를 확인해 보겠습니다.

In [ ]:
print(x_test.shape, y_test.shape)

테스트 세트에는 10,000개의 샘플이 포함되어 있습니다.

마지막으로 훈련 세트와 테스트 세트에 클래스가 골고루 포함되어 있는지 확이하겠습니다. 분류 작업은 어떤 샘플이 주어졌을 때 여러 개의 클래스 중 하나를 선택하는 문제입니다. 훈련 세트에 어떤 클래스의 데이터가 크게 부족하다면 그 클래스를 분류할 수 있는 능력을 학습하지 못할 것입니다. 마찬가지로 테스트 세트에 어떤 클래스가 부족하다면 모든 클래스에 대해서 성능을 공정하게 평가하지 못합니다.

타깃 데이터에서 레이블의 갯수를 세는 가장 편리한 방법은 넘파이 `bincount()` 함수를 사용하는 것입니다. 이 함수는 배열에 포함된 고유한 정수를 헤아려 정수 값에 해당하는 인덱스에 값을 저장합니다. 먼저 넘파이를 임포트합니다.

In [ ]:
import numpy as np

간단한 예를 통해 `bincount()` 함수의 작동 방식을 이해해 보겠습니다.

In [ ]:
np.bincount([1, 0, 1, 3])

0이 하나이고, 1은 두개, 3이 하나이므로 각각 인덱스 0, 인덱스 1, 인덱스 3에 카운트 값을 채웠습니다. 인덱스 2는 비워져 있습니다.

이제 `y_train`과 `y_test`이 가지고 있는 클래스 분포를 확인해 보겠습니다.

In [ ]:
np.bincount(y_train)

In [ ]:
np.bincount(y_test)

## 데이터 전처리

신경망에 사용할 데이터는 평균을 0, 분산을 1로 맞추는 것이 일반적입니다. 이미지 데이터는 255로 나누어 0~1 사이의 값을 갖도록 만듭니다. 이것이 표준화는 아니지만 잘 작동하기 때문에 많이 사용합니다.

In [ ]:
x_train_scaled = x_train / 255
x_test_scaled = x_test / 255

패션 MNIST 데이터셋은 흑백 이미지로 구성되어 있기 때문에 2차원 배열로도 픽셀 값을 표현할 수 있습니다. 하지만 합성곱 신경망은 채널 값이 있는 3차원 배열을 기대합니다. 따라서 28x28 크기의 2차원 배열을 28x28x1 크기의 3차원 배열로 변환하겠습니다. 

넘파이 배열의 `reshape()` 메서드는 배열의 총 원소 갯수를 동일하게 유지하면서 크기를 변경할 수 있습니다. 어떤 차원을 `-1`로 지정하면 해당 차원의 크기는 나머지 차원이 모두 결정되고 난 후 자동으로 결정된다는 의미입니다.

In [ ]:
x_train_scaled = x_train_scaled.reshape(-1, 28, 28, 1)
x_test_scaled = x_test_scaled.reshape(-1, 28, 28, 1)

## 모델 구성

합성곱 신경망을 구성하기 위해 `Sequential` 클래스의 객체를 만듭니다.

In [ ]:
model = keras.Sequential()

케라스 API에서 제공하는 합성곱 층은 `Conv2D`입니다. 합성곱 층에는 전형적으로 렐루(ReLU) 활성화 함수를 사용합니다.

합성곱의 수행 결과의 높이와 너비가 원본 입력보다 줄어들지 않게 하기 위해 입력 배열 주위에 0으로 패딩을 추가합니다. 이를 세임(SAME) 패딩이라 부릅니다. 또 다른 패딩은 밸리드(VALID) 패딩으로 제로(0) 패딩을 추가하지 않는 방법입니다. 합성곱 신경망은 일반적으로 세임 패딩을 사용하여 출력 특성 맵의 크기를 입력과 동일하게 유지시키고 풀링 단계에서 특성 맵의 크기를 줄입니다.

합성곱의 커널의 크기는 보통 3x3, 5x5 등을 많이 사용합니다. 여기에서는 10개의 커널을 사용하겠습니다.

In [ ]:
model.add(keras.layers.Conv2D(10, (3, 3), activation='relu', 
                              padding='same', input_shape=(28, 28, 1)))

합성곱 층 다음에 풀링 층을 추가합니다. 많이 사용하는 풀링 층은 맥스 풀링입니다. 케라스 API에서는 `MaxPooling2D`에 구현되어 있습니다.

풀링 층에서 사용할 스캔 영역의 크기는 전형적으로 2x2 크기입니다. 이전 특성 맵의 각 2x2 크기의 영역에서 가장 큰 값을 골라 다음 특성 맵을 만듭니다.

In [ ]:
model.add(keras.layers.MaxPooling2D((2, 2)))

10개의 의류를 분류하는 문제이므로 마지막 출력 층은 10개의 뉴런으로 구성되어야 합니다. 이 뉴런의 출력을 각 클래스 레이블의 확률값으로 해석할 것입니다.

풀링 층에서 출력된 샘플 하나의 특성 맵은 3차원으로 구성되어 있습니다. 이를 완전 연결 층의 입력으로 사용하려면 1열로 펼쳐야 합니다. 이런 작업을 하는 층이 `Flatten` 층입니다. 이 층은 똑똑하게도 첫 번째 배치 차원은 유지하고 나머지 차원을 모두 펼쳐 줍니다.

In [ ]:
model.add(keras.layers.Flatten())

일반적으로 최종 출력 층으로 바로 연결하기 전에 한 개 이상의 완전 연결층을 둡니다. 여기에서도 100개의 뉴런을 가진 완전 연결층과 10개의 뉴런을 가진 출력층을 추가하겠습니다.

완전 연결층에는 합성곱 신경망과 마찬가지로 렐루 활성화 함수를 사용합니다. 2개 이상의 클래스를 분류하는 다중 분류 신경망의 출력층에는 소프트맥스 함수를 사용합니다. 소프트맥스 함수는 출력층에 있는 뉴런의 출력 값을 정규화하여 전체 합이 1이 되도록 만들어 확률로 이해하기 쉽습니다.

In [ ]:
model.add(keras.layers.Dense(100, activation='relu'))
model.add(keras.layers.Dense(10, activation='softmax'))

모델 구성을 마쳤습니다. `summary()` 메서드로 전체 모델의 구조를 검토해 보겠습니다.

In [ ]:
model.summary()

첫 번째 합성곱 층에서 세임 패딩을 사용했기 때문에 출력 특성 맵의 높이와 너비는 변하지 않고 28x28 크기를 유지했습니다. 이 합성곱 층에서 사용한 커널의 개수는 10개이므로 특성 맵의 채널 수가 10으로 늘어났습니다.

두 번째 풀링 층은 채널 별로 적용되기 때문에 채널의 크기는 그대로 유지됩니다. 풀링의 크기를 2x2로 지정했기 때문에 특성 맵의 높이와 너비가 절반으로 줄어 들엇습니다.

그 다음 플래튼 층은 14x14x10 크기의 특성 맵을 펼쳐서 1,960 크기의 벡터로 만듭니다.

이어지는 두 개의 완전 연결 층은 각기 100개의 뉴런과 10개의 뉴런으로 출력의 크기를 바꾸었습니다.

이 네트워크에서 사용하는 가중치 크기를 확인해 보겠습니다. 합성곱 층에서는 3x3 크기의 커널이 10개 사용되었습니다. 커널마다 절편이 하나씩있으므로 합성곱에서 사용하는 가중치 갯수는 다음과 같습니다.

$3 \times 3 \times 10 + 10 = 100$

풀링 층과 플래튼 층은 학습되는 가중치가 없습니다.

그다음 완전 연결 층은 100개의 뉴런이 1960개의 입력과 연결되고 역시 100개의 절편이 있습니다.

$1960 \times 100 + 100 = 196100$

마지막 출력층은 10개의 뉴런이 100개의 입력과 연결되고 10개의 절편이 있습니다.

$100 \times 10 + 10 = 1010$

각 층의 가중치 갯수를 확인해 보면 합성곱 층이 완전 연결 층보다 월등히 작은 것을 알 수 있습니다. 합성곱 층에서는 커널 하나가 전체 입력에 동일하게 사용되기 때문입니다. 이로 인해 합성곱 층에서는 여러 개의 커널을 사용해도 부담이 작습니다.

## 모델 훈련 및 테스트

이제 모델에 옵티마이저와 손실 함수를 연결할 차례입니다. 여기에서도 아담 옵티마이저를 사용하겠습니다. 회귀 문제에서는 `'mae'` 손실 함수를 사용했습니다. 분류 작업에서는 크로스엔트로피 손실 함수를 사용합니다. 크로스엔트로피 공식은 다음과 같이 정의됩니다.

$L = \sum_{i=1}^{C}y_i \text{log} a_i$

여기에서 C는 클래스 갯수입니다. a는 모델의 출력값이고 y는 타깃입니다.

케라스 API에서는 이진 분류(binary classification)일 경우 `'binary_crossentropy'`를 지정하고 다중 분류(multiclass classification)일 경우 `'categorical_crossentropy'`를 지정합니다. 만약 다중 분류에서 타깃 값이 원-핫 인코딩이 아니고 레이블을 나타내는 정수 값일 때는 `'sparse_categorical_crossentropy'`로 지정해야 합니다.

회귀 문제에서는 훈련 과정에서 MAE 손실 값만 기록했습니다. 여기에서는 크로스엔트로피 손실과 함께 정확도를 기록해 보겠습니다. 정확도는 올바르게 분류된 샘플의 비율입니다. 정확도를 기록하려면 `compile()` 메서드의 `metrics` 매개변수에 `'acc'`를 리스트에 포함하여 전달해야 합니다.

In [ ]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy',
              metrics=['acc'])

`fit()` 메서드를 사용하여 모델을 훈련하겠습니다. 10번 에포크 동안 훈련하고 손실과 정확도 점수를 확인해 보죠

In [ ]:
history = model.fit(x_train_scaled, y_train, epochs=10)

에포크마다 손실 점수가 감소하고 정확도 점수가 증가합니다. 손실과 정확도가 정확히 선형적으로 반비례하지는 않습니다. 하지만 일반적인 경향은 손실이 감소하면 정확도가 증가합니다.

맷플롯립으로 손실 점수와 정확도를 출력해 보겠습니다. 정확도는 `history` 객체의 `history` 속성에 `'acc'`로 저장되어 있습니다.

In [ ]:
plt.plot(history.history['loss'])
plt.show()

In [ ]:
plt.plot(history.history['acc'])
plt.show()

테스트 샘플을 사용해 훈련된 모델이 만든 예측을 확인해 보죠. `predict()` 메서드는 마지막 출력층의 값을 반환합니다. 출력 층의 뉴런이 10개이므로 10개의 확률 값이 출력됩니다. 이 값의 총 합은 1입니다.

`predict()` 메서드는 입력 배열의 첫 번째 차원이 배치 차원일 것으로 기대합니다. 이 예제의 샘플은 28x28x1 크기이지만 하나의 샘플을 주입하더라도 1x28x28x1 크기로 전달해야 합니다. 이를 위해 파이썬의 슬라이싱 연산자를 사용하여 첫 번째 샘플을 전달합니다.

In [ ]:
model.predict(x_test_scaled[0:1])

위의 `predict()` 메서드의 결과 중에서 가장 큰 값은 마지막 원소입니다. 하지만 매번 이렇게 찾는 것은 번거롭습니다. 대신 `predict_classes()` 메서드를 사용하면 가장 높은 확률 값을 가진 인덱스를 반환합니다.

In [ ]:
preds = model.predict_classes(x_test_scaled[0:5])
print(preds)

이 모델의 예측 인덱스 `preds`를 `y_test`와 비교하여 예측이 정확히 맞았는지 확인할 수 있습니다.

In [ ]:
print(y_test[0:5])

또한 `preds`를 클래스 이름을 담은 리스트의 인덱스로 사용하여 실제 이미지와 클래스 레이블을 함께 출력할 수도 있습니다.

In [ ]:
for i in range(5):
    plt.imshow(x_test[i], cmap='gray')
    plt.show()
    print(class_names[preds[i]])